In [9]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('input_image.jpg')

# 1. Resize the image to a standard size
standard_size = (256, 256)
resized_image = cv2.resize(image, standard_size)

# 2. Convert the image to HSV color space
hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)

# 3. Apply Gaussian Blur to remove noise
blurred_image = cv2.GaussianBlur(hsv_image, (5, 5), 0)

# 4. Apply a Laplacian filter to enhance edges
laplacian_image = cv2.Laplacian(blurred_image, cv2.CV_64F)

# 5. Perform histogram equalization on the V channel (intensity)
hsv_image[:, :, 2] = cv2.equalizeHist(hsv_image[:, :, 2])
equalized_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

# 6. Perform background removal (example using simple thresholding)
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray_image, 100, 255, cv2.THRESH_BINARY)
background_removed_image = cv2.bitwise_and(resized_image, resized_image, mask=mask)

# Save the preprocessed image
cv2.imwrite('preprocessed_image.jpg', background_removed_image)

# Display the images
cv2.imshow('Original Image', image)
cv2.imshow('Resized Image', resized_image)
cv2.imshow('HSV Image', hsv_image)
cv2.imshow('Blurred Image', blurred_image)
cv2.imshow('Laplacian Image', laplacian_image)
cv2.imshow('Equalized Image', equalized_image)
cv2.imshow('Background Removed Image', background_removed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [13]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('input_image.jpg')

# 1. Convert image to a specific format (e.g., RGB)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 2. Resize the image to a standard size (e.g., 256x256)
standard_size = (256, 256)
resized_image = cv2.resize(image_rgb, standard_size)

# 3. Convert the image to HSV color space
hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2HSV)

# 4. Convert to HSI color space (approximated using HSV as OpenCV does not directly support HSI)
h, s, v = cv2.split(hsv_image)
i = (2*v + s) // 3  # Approximating Intensity for HSI
hsi_image = cv2.merge([h, s, i])

# 5. Noise removal using Gaussian Blur
blurred_image = cv2.GaussianBlur(hsi_image, (5, 5), 0)

# 6. Background removal using masking (white background assumption)
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY)
_, mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
background_removed_image = cv2.bitwise_and(resized_image, resized_image, mask=~mask)

# 7. Low-pass filter to reduce high-frequency noise
low_pass = cv2.GaussianBlur(blurred_image, (5, 5), 0)

# 8. High-pass filter to enhance features (edges)
high_pass = cv2.Laplacian(blurred_image, cv2.CV_64F)

# 9. Convert the high-pass image to the same type as the low-pass image
high_pass = cv2.convertScaleAbs(high_pass)

# 10. Combine low-pass and high-pass filters for feature enhancement
enhanced_image = cv2.addWeighted(low_pass, 0.5, high_pass, 0.5, 0)

# 11. Histogram equalization for contrast enhancement
# Histogram equalization on the intensity channel (I)
i_eq = cv2.equalizeHist(i)
enhanced_hsi_image = cv2.merge([h, s, i_eq])
equalized_image = cv2.cvtColor(enhanced_hsi_image, cv2.COLOR_HSV2RGB)

# Save the preprocessed image
cv2.imwrite('preprocessed_image.jpg', equalized_image)

# Display the original and processed images
cv2.imshow('Original Image', image)
cv2.imshow('Resized Image', resized_image)
cv2.imshow('HSV Image', hsv_image)
cv2.imshow('HSI Image', hsi_image)
cv2.imshow('Blurred Image', blurred_image)
cv2.imshow('Background Removed Image', background_removed_image)
cv2.imshow('Enhanced Image', enhanced_image)
cv2.imshow('Equalized Image', equalized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [23]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('input_image.jpg')

# 1. Convert image to a specific format (e.g., RGB)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 2. Resize the image to a standard size (e.g., 256x256)
standard_size = (256, 256)
resized_image = cv2.resize(image_rgb, standard_size)

# 3. Convert the image to HSV color space
hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2HSV)

# 4. Convert to HSI color space (approximated using HSV as OpenCV does not directly support HSI)
h, s, v = cv2.split(hsv_image)
i = (2*v + s) // 3  # Approximating Intensity for HSI
hsi_image = cv2.merge([h, s, i])

# 5. Noise removal using Gaussian Blur
blurred_image = cv2.GaussianBlur(hsi_image, (5, 5), 0)

# 6. Background removal using adaptive thresholding and morphological operations
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY)
blurred_gray = cv2.GaussianBlur(gray_image, (5, 5), 0)
adaptive_thresh = cv2.adaptiveThreshold(blurred_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                        cv2.THRESH_BINARY_INV, 11, 2)

# Use morphological operations to clean up the mask
kernel = np.ones((3, 3), np.uint8)
cleaned_mask = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

# Apply the mask to remove the background
background_removed_image = cv2.bitwise_and(resized_image, resized_image, mask=cleaned_mask)

# 7. Low-pass filter to reduce high-frequency noise
low_pass = cv2.GaussianBlur(blurred_image, (5, 5), 0)

# 8. High-pass filter to enhance features (edges)
high_pass = cv2.Laplacian(blurred_image, cv2.CV_64F)

# 9. Convert the high-pass image to the same type as the low-pass image
high_pass = cv2.convertScaleAbs(high_pass)

# 10. Combine low-pass and high-pass filters for feature enhancement
enhanced_image = cv2.addWeighted(low_pass, 0.5, high_pass, 0.5, 0)

# 11. Histogram equalization for contrast enhancement
# Histogram equalization on the intensity channel (I)
i_eq = cv2.equalizeHist(i)
enhanced_hsi_image = cv2.merge([h, s, i_eq])
equalized_image = cv2.cvtColor(enhanced_hsi_image, cv2.COLOR_HSV2RGB)

# Save the preprocessed image
cv2.imwrite('preprocessed_image.jpg', equalized_image)
cv2.imwrite('preprocessed_image.jpg', equalized_image)

# Display the original and processed images
cv2.imshow('Original Image', image)
cv2.imshow('Resized Image', resized_image)
cv2.imshow('HSV Image', hsv_image)
cv2.imshow('HSI Image', hsi_image)
cv2.imshow('Blurred Image', blurred_image)
cv2.imshow('Background Removed Image', background_removed_image)
cv2.imshow('Enhanced Image', enhanced_image)
cv2.imshow('Equalized Image', equalized_image)
cv2.waitKey(0)
import time
time.sleep(10)
cv2.destroyAllWindows()


In [21]:

cv2.destroyAllWindows()

In [28]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('real.jpg')

# 1. Convert image to a specific format (e.g., RGB)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 2. Resize the image to a standard size (e.g., 256x256)
standard_size = (256, 256)
resized_image = cv2.resize(image_rgb, standard_size)

# 3. Convert the image to HSV color space
hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2HSV)

# 4. Convert to HSI color space (approximated using HSV as OpenCV does not directly support HSI)
h, s, v = cv2.split(hsv_image)
i = (2*v + s) // 3  # Approximating Intensity for HSI
hsi_image = cv2.merge([h, s, i])

# 5. Noise removal using Gaussian Blur
blurred_image = cv2.GaussianBlur(hsi_image, (5, 5), 0)

# 6. Improved background removal using GrabCut
mask = np.zeros(resized_image.shape[:2], np.uint8)

# Rect should cover the entire object
rect = (10, 10, resized_image.shape[1] - 10, resized_image.shape[0] - 10)

# GrabCut algorithm requires two arrays to be initialized
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

# Apply GrabCut
cv2.grabCut(resized_image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

# The mask is now marked with 0 (background), 1 (probable background), 
# 2 (probable foreground), and 3 (foreground)
mask2 = np.where((mask == 2) | (mask == 3), 255, 0).astype('uint8')

# Use the mask to extract the foreground
background_removed_image = cv2.bitwise_and(resized_image, resized_image, mask=mask2)

# 7. Low-pass filter to reduce high-frequency noise
low_pass = cv2.GaussianBlur(blurred_image, (5, 5), 0)

# 8. High-pass filter to enhance features (edges)
high_pass = cv2.Laplacian(blurred_image, cv2.CV_64F)

# 9. Convert the high-pass image to the same type as the low-pass image
high_pass = cv2.convertScaleAbs(high_pass)

# 10. Combine low-pass and high-pass filters for feature enhancement
enhanced_image = cv2.addWeighted(low_pass, 0.5, high_pass, 0.5, 0)

# 11. Histogram equalization for contrast enhancement
# Histogram equalization on the intensity channel (I)
i_eq = cv2.equalizeHist(i)
enhanced_hsi_image = cv2.merge([h, s, i_eq])
equalized_image = cv2.cvtColor(enhanced_hsi_image, cv2.COLOR_HSV2RGB)

# Save the preprocessed image
cv2.imwrite('preprocessed_image.jpg', equalized_image)

# Display the original and processed images
cv2.imshow('Original Image', image)
cv2.imshow('Resized Image', resized_image)
cv2.imshow('HSV Image', hsv_image)
cv2.imshow('HSI Image', hsi_image)
cv2.imshow('Blurred Image', blurred_image)
cv2.imshow('Background Removed Image', background_removed_image)
cv2.imshow('Enhanced Image', enhanced_image)
cv2.imshow('Equalized Image', equalized_image)
cv2.waitKey(0)
import time
time.sleep(10)
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [29]:

cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np

# Load the image
# image = cv2.imread('real.jpg')


def preprocess_image(image):
	# 1. Convert image to a specific format (e.g., RGB)
	image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	# image_rgb = image

	# 2. Resize the image to a standard size (e.g., 256x256)
	standard_size = (256, 256)
	resized_image = cv2.resize(image_rgb, standard_size)



	# import os,sys
	# current_dir =os.getcwd()
	# Add the module's parent directory to PYTHONPATH
	# sys.path.append(os.path.join(current_dir, 'leafimagesegmentation'))
	from segment import segment_leaf_mine
	_,bg_rm_img = segment_leaf_mine(resized_image, 'FLOOD', False, 0)

	gray_image = cv2.cvtColor(bg_rm_img, cv2.COLOR_RGB2GRAY)

	# Apply a binary threshold to create a mask
	_, binary_mask = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

	# Find contours in the binary mask
	contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

	# Get the bounding box of the largest contour (assuming it's the colored part)
	x, y, w, h = cv2.boundingRect(contours[0])

	# Crop the image to the bounding box
	cropped_image = bg_rm_img[y:y+h, x:x+w]

	cv2.imwrite('cropped_image.jpg', cropped_image)


	# 3. Convert the image to HSV color space
	hsv_image = cv2.cvtColor(cropped_image, cv2.COLOR_RGB2HSV)

	# 4. Convert to HSI color space (approximated using HSV as OpenCV does not directly support HSI)
	h, s, v = cv2.split(hsv_image)
	i = (2*v + s) // 3  # Approximating Intensity for HSI
	hsi_image = cv2.merge([h, s, i])

	# 5. Noise removal using Gaussian Blur
	blurred_image = cv2.GaussianBlur(hsi_image, (5, 5), 0)


	# 6. Low-pass filter to reduce high-frequency noise
	low_pass = cv2.GaussianBlur(blurred_image, (5, 5), 0)

	# 7. High-pass filter to enhance features (edges)
	high_pass = cv2.Laplacian(blurred_image, cv2.CV_64F)

	# 8. Convert the high-pass image to the same type as the low-pass image
	high_pass = cv2.convertScaleAbs(high_pass)

	# 9. Combine low-pass and high-pass filters for feature enhancement
	enhanced_image = cv2.addWeighted(low_pass, 0.5, high_pass, 0.5, 0)

	# 10. Histogram equalization for contrast enhancement
	# Histogram equalization on the intensity channel (I)
	i_eq = cv2.equalizeHist(i)
	enhanced_hsi_image = cv2.merge([h, s, i_eq])
	equalized_image = cv2.cvtColor(enhanced_hsi_image, cv2.COLOR_HSV2RGB)

	# Save the preprocessed image
	cv2.imwrite('preprocessed_image.jpg', equalized_image)

	# Display the original and processed images
	cv2.imshow('Original Image', image)
	cv2.imshow('Resized Image', resized_image)
	cv2.imshow('HSV Image', hsv_image)
	cv2.imshow('HSI Image', hsi_image)
	cv2.imshow('Blurred Image', blurred_image)
	cv2.imshow('BG remove Image', bg_rm_img)
	cv2.imshow('Enhanced Image', enhanced_image)
	cv2.imshow('Equalized Image', equalized_image)
	cv2.waitKey(0)
	import time
	time.sleep(2)
	cv2.destroyAllWindows()


In [4]:
import os
# for img in os.listdir("./imgs"):
image = cv2.imread(os.path.join("imgs","1.jpg"))
preprocess_image(image)

In [5]:
import os
os.listdir("preprocessed_data_test")

['Soybean___healthy',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Corn___healthy',
 'Potato___Early_blight',
 'Apple___Black_rot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Corn___Northern_Leaf_Blight',
 'Tomato___Tomato_mosaic_virus',
 'Potato___healthy',
 'Strawberry___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Potato___Late_blight',
 'Raspberry___healthy',
 'Apple___Cedar_apple_rust',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Target_Spot',
 'Tomato___Leaf_Mold',
 'Grape___Esca_(Black_Measles)',
 'Tomato___Late_blight',
 'Apple___Apple_scab',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Cherry___Powdery_mildew',
 'Peach___healthy',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Blueberry___healthy',
 'Tomato___healthy',
 'Strawberry___Leaf_scorch',
 'Grape___Black_rot',
 'Tomato___Early_blight',
 'Cherry___healthy',
 'Corn___Common_rust',
 'Background_without_leaves',
 'Pepper,_bell___Bacterial_spot',
 'Apple___healthy',
 'Squash___